In [ ]:
# some directory for output the results:
OUTPUT_PATH = '/content/drive/MyDrive/Preprocessed_Luna_2/'

# Resource path which contains: annotations.csv, candidates.csv,
# and subdirectories containing .mhd files.
# This is the directory structure needed to run the code:
# (The code will use all .mhd and .raw files inside subdirectories which their name is in annotations or candidates)
'''
[RESOURCES_PATH]/
            annotations.csv
            candidates.csv
            subset0/
                        *.mhd
                        *.raw
            subset1/
                        *.mhd
                        *.raw
            my_custom_subset/
                        *.mhd
                        *.raw
'''
RESOURCES_PATH = '/content/drive/MyDrive/Luna/'





PADDING_FOR_LOCALIZATION = 10
BLOCK_SIZE = 128
COORDS_CUBE_SIZE = 32
TARGET_SHAPE = (COORDS_CUBE_SIZE, COORDS_CUBE_SIZE, COORDS_CUBE_SIZE, 3, 5)
COORDS_SHAPE = (3, COORDS_CUBE_SIZE, COORDS_CUBE_SIZE, COORDS_CUBE_SIZE)
ANCHOR_SIZES = [10, 30, 60]
VAL_PCT = 0.2
TOTAL_EPOCHS = 100
DEFAULT_LR = 0.01
ANNOTATION_EXIST = True

In [ ]:


# UTILITY

from functools import partial

import numpy as np
from matplotlib import pyplot as plt
from scipy import ndimage as ndi
import scipy
from skimage.filters import roberts
from skimage.measure import label, regionprops
from skimage.morphology import convex_hull_image, disk, binary_closing
from skimage.segmentation import clear_border


def argmax_3d(img: np.array):
    max1 = np.max(img, axis=0)
    argmax1 = np.argmax(img, axis=0)
    max2 = np.max(max1, axis=0)
    argmax2 = np.argmax(max1, axis=0)
    argmax3 = np.argmax(max2, axis=0)
    argmax_3d = (argmax1[argmax2[argmax3], argmax3], argmax2[argmax3], argmax3)
    return argmax_3d, img[argmax_3d]


def get_segmented_lungs(im, plot=False):
    '''
    This funtion segments the lungs from the given 2D slice.
    '''
    plt_number = 0
    # Original image label: 0
    if plot:
        f, plots = plt.subplots(12, 1, figsize=(5, 40))
        plots[plt_number].axis('off')
        plots[plt_number].set_title(f'{plt_number}')
        plots[plt_number].imshow(im, cmap=plt.cm.bone)
        plt_number += 1

    # Step 1: Convert into a binary image.
    # image label: 1
    binary = im < -604
    cleared = clear_border(binary)
    label_image = label(cleared)
    areas = [r.area for r in regionprops(label_image)]
    areas.sort()
    labels = []
    if len(areas) > 2:
        for region in regionprops(label_image):
            if region.area < areas[-2]:
                for coordinates in region.coords:
                    label_image[coordinates[0], coordinates[1]] = 0
            else:
                coordinates = region.coords[0]
                labels.append(label_image[coordinates[0], coordinates[1]])
    else:
        labels = [1, 2]
    rig = label_image == labels[0]
    lef = label_image == labels[1]
    r_edges = roberts(rig)
    l_edges = roberts(lef)
    rig = ndi.binary_fill_holes(r_edges)
    lef = ndi.binary_fill_holes(l_edges)

    rig = convex_hull_image(rig)
    lef = convex_hull_image(lef)

    sum_of_lr = rig + lef
    binary = sum_of_lr > 0

    selem = disk(10)
    binary = binary_closing(binary, selem)

    # Step 9: Superimpose the binary mask on the input image.
    # image label: 11
    get_high_vals = binary == 0
    im[get_high_vals] = 0
    if plot:
        plots[plt_number].axis('off')
        plots[plt_number].set_title(f'{plt_number}')
        plots[plt_number].imshow(im, cmap=plt.cm.bone)
        plt_number += 1


    return im, convex_hull_image(binary)



# Show the 3D Scan

import matplotlib.pyplot as plt
from matplotlib.widgets import Slider
import numpy as np



# CT Scan Utility
import scipy.misc
import numpy as np
import SimpleITK as sitk
from glob import glob
from skimage.measure import regionprops


class CTScan(object):
    def __init__(self, seriesuid, centers=None, radii=None, clazz=None):
        self._seriesuid = seriesuid
        self._centers = centers
        paths = glob(f'''{RESOURCES_PATH}/*/{self._seriesuid}.mhd''')
        path = paths[0]
        self._ds = sitk.ReadImage(path)
        self._spacing = np.array(list(reversed(self._ds.GetSpacing())))
        self._origin = np.array(list(reversed(self._ds.GetOrigin())))
        self._image = sitk.GetArrayFromImage(self._ds)
        self._radii = radii
        self._clazz = clazz
        self._mask = None

    def preprocess(self, info=True):
        if(info):
          print("Resampling")
        self._resample()
        if(info):
          print("Segmenting Lung")
        self._segment_lung_from_ct_scan()
        if(info):
          print("Normalizing")
        self._normalize()
        if(info):
          print("Zero Centering")
        self._zero_center()

        if(ANNOTATION_EXIST):
          if(info):
            print("Changing Coords")
          self._change_coords()

    def save_preprocessed_image(self, plot=False):
        if(self._clazz == None):
          subdir = 'unlabeled'
          file_path = f'''preprocessed/{subdir}/{self._seriesuid}.npy'''
          print("Saving to file_path")
          np.save(f'{OUTPUT_PATH}/{file_path}', self._image)
        else:
          subdir = 'negatives' if self._clazz == 0 else 'positives'
          file_path = f'''preprocessed/{subdir}/{self._seriesuid}.npy'''
          np.save(f'{OUTPUT_PATH}/{file_path}', self._image)

        if(plot== True):
          show_3d_numpy_array(self._image)

    def get_info_dict(self):
        (min_z, min_y, min_x, max_z, max_y, max_x) = (None, None, None, None, None, None)
        for region in regionprops(self._mask):
            min_z, min_y, min_x, max_z, max_y, max_x = region.bbox
        assert (min_z, min_y, min_x, max_z, max_y, max_x) != (None, None, None, None, None, None)
        min_point = (min_z, min_y, min_x)
        max_point = (max_z, max_y, max_x)
        return {'seriesuid': self._seriesuid, 'radii': self._radii, 'centers': self._centers,
                'spacing': list(self._spacing), 'lungs_bounding_box': [min_point, max_point], 'class': self._clazz}

    def _resample(self):
        spacing = np.array(self._spacing, dtype=np.float32)
        new_spacing = [1, 1, 1]
        imgs = self._image
        new_shape = np.round(imgs.shape * spacing / new_spacing)
        true_spacing = spacing * imgs.shape / new_shape
        resize_factor = new_shape / imgs.shape
        imgs = scipy.ndimage.zoom(imgs, resize_factor, mode='nearest')
        self._image = imgs
        self._spacing = true_spacing

    def _segment_lung_from_ct_scan(self):
        result_img = []
        result_mask = []
        num = 0
        for slicee in self._image:
            rimg, rmsk = get_segmented_lungs(slicee)
            result_img.append(rimg)
            result_mask.append(rmsk)
        self._image = np.asarray(result_img)
        self._mask = np.asarray(result_mask, dtype=int)

    def _world_to_voxel(self, worldCoord):
        stretchedVoxelCoord = np.absolute(np.array(worldCoord) - np.array(self._origin))
        voxelCoord = stretchedVoxelCoord / np.array(self._spacing)
        return voxelCoord.astype(int)

    def _get_world_to_voxel_coords(self, idx):
        return tuple(self._world_to_voxel(self._centers[idx]))

    def _get_voxel_coords(self):
        voxel_coords = [self._get_world_to_voxel_coords(j) for j in range(len(self._centers))]
        return voxel_coords

    def _change_coords(self):
        new_coords = self._get_voxel_coords()
        self._centers = new_coords

    def _normalize(self):
        MIN_BOUND = -1200
        MAX_BOUND = 600.
        self._image = (self._image - MIN_BOUND) / (MAX_BOUND - MIN_BOUND)
        self._image[self._image > 1] = 1.
        self._image[self._image < 0] = 0.
        self._image *= 255.

    def _zero_center(self):
        PIXEL_MEAN = 0.25 * 256
        self._image = self._image - PIXEL_MEAN




# RUN PREPROCESS
import pandas as pd
import numpy as np
from glob import glob
import os

if(ANNOTATION_EXIST):
  annotations = pd.read_csv(RESOURCES_PATH + '/annotations.csv')
  candidates = pd.read_csv(RESOURCES_PATH + '/candidates.csv')


def _get_positive_series():
    paths = glob(RESOURCES_PATH + '\\*\\' + "*.mhd")
    file_list = [f.split('\\')[-1][:-4] for f in paths]
    series = annotations['seriesuid'].tolist()
    infected = [f for f in file_list if f in series]
    return infected


def _get_negative_series():
    paths = glob(RESOURCES_PATH + '\\*\\' + "*.mhd")
    file_list = [f.split('\\')[-1][:-4] for f in paths]
    series = annotations['seriesuid'].tolist()
    cleans = [f for f in file_list if f not in series]
    return cleans

def _get_unlabeled_series():
    paths = glob(RESOURCES_PATH + '\\*\\' + "*.mhd")
    file_list = [f.split('\\')[-1][:-4] for f in paths]
    # print("File list: ",file_list)
    return file_list


def save_preprocessed_data():
    if(ANNOTATION_EXIST):
        print("Preprocessing With Annotation")
        [os.makedirs(d, exist_ok=True) for d in
        [f'{OUTPUT_PATH}/preprocessed/positives', f'{OUTPUT_PATH}/preprocessed/negatives']]
        meta_data = pd.DataFrame(columns=['seriesuid', 'spacing', 'lungs_bounding_box', 'centers', 'radii', 'class'])


        total_length_pos = len(_get_positive_series())
        total_length_neg = len(_get_negative_series())

        processed_num = 0
        neg_processed_num = 0

        i = 0
        preprocess_num = 20
        for series_id in _get_positive_series():
            print("Processing id pos: ",series_id, " num-left: ", total_length_pos - processed_num)
            processed_num += 1

            nodule_coords_annot = annotations[annotations['seriesuid'] == series_id]
            tp_co = [(a['coordZ'], a['coordY'], a['coordX']) for a in nodule_coords_annot.iloc]
            radii = [(a['diameter_mm'] / 2) for a in nodule_coords_annot.iloc]
            ct = CTScan(seriesuid=series_id, centers=tp_co, radii=radii, clazz=1)
            ct.preprocess()
            ct.save_preprocessed_image()
            diction = ct.get_info_dict()
            print("diction: ",diction)
            meta_data.loc[len(meta_data)] = pd.Series(diction)
            # meta_data = pd.concat([meta_data, pd.Series(diction)], ignore_index=True)

            if(i > preprocess_num):
                print("Stopping")
                break
            else:
                i+=1

        for series_id in _get_negative_series():
            print("Processing id neg: ",series_id, " num-left: ", total_length_neg - neg_processed_num)
            neg_processed_num += 1

            nodule_coords_candid = candidates[candidates['seriesuid'] == series_id]
            tp_co = [(a['coordZ'], a['coordY'], a['coordX']) for a in nodule_coords_candid.iloc]
            radii = list(np.random.randint(40, size=len(tp_co)))
            max_numbers_to_use = min(len(tp_co), 3)
            tp_co = tp_co[:max_numbers_to_use]
            radii = radii[:max_numbers_to_use]
            ct = CTScan(seriesuid=series_id, centers=tp_co, radii=radii, clazz=0)
            ct.preprocess()
            ct.save_preprocessed_image()
            diction = ct.get_info_dict()
            meta_data.loc[len(meta_data)] = pd.Series(diction)


        meta_data.to_csv(f'{OUTPUT_PATH}/preprocessed_meta.csv')





    else:
        print("Preprocessing Without Annotation")

        [os.makedirs(d, exist_ok=True) for d in
        [f'{OUTPUT_PATH}/preprocessed/unlabeled']]

        meta_data = pd.DataFrame(columns=['seriesuid', 'spacing', 'lungs_bounding_box', 'centers', 'radii', 'class'])


        for series_id in _get_unlabeled_series():
            print("Processing id unlabeled: ",series_id)
            ct = CTScan(seriesuid=series_id,centers=[(0,0,0)], radii=[0], clazz=2)
            ct.preprocess()
            ct.save_preprocessed_image()

            diction = ct.get_info_dict()
            meta_data.loc[len(meta_data)] = pd.Series(diction)
            break

        meta_data.to_csv(f'{OUTPUT_PATH}/preprocessed_meta_unlabeled.csv')





if __name__ == '__main__':
    save_preprocessed_data()